# Mounting Drive, Importing Libraries and Algorithm

In [ ]:
#Mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#Clone SORT
!git clone https://github.com/abewley/sort 

Cloning into 'sort'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 202 (delta 0), reused 1 (delta 0), pack-reused 198
Receiving objects: 100% (202/202), 1.21 MiB | 1.63 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [ ]:
!python -m pip install filterpy

     |████████████████████████████████| 184kB 2.9MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp37-none-any.whl size=110476 sha256=4cae3e48a946f2beb45675c9fe76065af6d650dcc354ddfc134565e2dfadb66c
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
Successfully built filterpy


In [ ]:
#Import library
from sort.sort import *
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import random
import imutils

In [ ]:
#Get colors for bounding boxes
colors = []
for j in range(3000): colors.append(((int)(random.randrange(255)), (int)(random.randrange(255)), (int)(random.randrange(255))))

In [ ]:
mot_tracker = Sort()

#Set size
dim = (640, 640)

#Set counter
n = 1
for n in range(1, 2452):
  #Read image
  cap = cv2.VideoCapture('/content/drive/MyDrive/images/svo_oct05-04_Trim ' + str(n).zfill(4) + '.jpg')
  ret, img = cap.read()

  #Resize image
  img = cv2.resize(img, (640, 640))

  #Try to read the image if there is a bounding box. If not, read the empty file.
  try:
    #Read coordinates of bounding boxes
    coords = open('/content/drive/MyDrive/labels/content/yolov5/runs/detect/out_trim3/labels/svo_oct05-04_Trim_' + str(n) + '.txt')
  except:
    #No coordinates available
    coords = open('/content/drive/MyDrive/labels/content/yolov5/runs/detect/out_trim3/labels/empty.txt')

# Stripping data from the txt file into a list #
  list_of_lists = []
  for line in coords:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    list_of_lists.append(line_list)
  coords.close()

# Conversion of str to int #
  stage1 = []
  for i in range(0, len(list_of_lists)):
    test_list = list(map(float, list_of_lists[i])) 
    stage1.append(test_list)


# Denormalize coordinates
  stage2 = []
  try:
    h,w,_ = img.shape
  except:
    continue
  mul = [1,w,h,w,h]
  for x in stage1:
    xx, yy, w, h, c =  x[1] * mul[1], x[2]*mul[2], x[3]*mul[3], x[4]*mul[4] , 0.4  
    stage2.append([xx,yy,w,h,c])

    #Check for incidents
    if (yy > 548) or (yy < 280):
      print('INCIDENT HAPPENED')


# Convert (x_center, y_center, width, height) --> (x_min, y_min, width, height)
  stage_final = []
  for x in stage2:
    xx,yy,w,h,c =  (x[0]-(x[2]/2)) , (x[1]-(x[3]/2)), x[2]*1, x[3]*1, x[4] 
    x1, y1, x2, y2, c = xx, yy, xx + w, yy + h, c
    stage_final.append([x1, y1, x2, y2, c])


  result_img = np.copy(img)
  if len(stage_final)!=0:
    stage_final = np.array(stage_final)
    trackers = mot_tracker.update(stage_final)
    for d in trackers:
      result_img = cv2.rectangle(result_img, ((int)(d[0]), (int)(d[1])), ((int)(d[2]), (int)(d[3])), color_list[(int)(d[4])], 2)
      result_img = cv2.putText(result_img, ((str)(d[4])), ((int)(d[0]), (int)(d[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color_list[(int)(d[4])], 2)

  #Save the result to the Google Drive
  name = ('/content/drive/MyDrive/tracking_final/track' + str(n) + '.jpg')
  cv2.imwrite(name, result_img)
  cv2.waitKey(1)